# Pull in the transliteration table

In [3]:
import requests

user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'
headers = {'User-Agent': user_agent}
response = requests.get('https://scriptsource.org/cms/scripts/page.php?item_id=entry_detail&uid=vsytndbyev', headers=headers)

In [230]:
import pandas as pd

results = pd.read_html(response.text)
df = results[1]
df.head()

,Glyph,USV,ALA-LC,Alone-Stokes,B&D,Campbell,Chaîne,Cohen,Dawkins,Dept of State,ethiop,SERA
0,NaN,1200,ha,hā,hä,ha,hă,ha,hä,ha,ha,he
1,NaN,1201,hu,hu,hu,hū,hu,hu,hu,hu,hu,hu
2,NaN,1202,hi,hī,hi,hī,hi,hi,hi,hi,hi,hi
3,NaN,1203,hā,hā,ha,hā,ha,ha,ha,ha,hā,ha
4,NaN,1204,hé,hē,he,hē,he,he,he,he,hē,hE


In [24]:
df.Glyph = df.USV.apply(lambda val: chr(int(val, 16)))
df.head()

,Glyph,USV,ALA-LC,Alone-Stokes,B&D,Campbell,Chaîne,Cohen,Dawkins,Dept of State,ethiop,SERA
0,ሀ,1200,ha,hā,hä,ha,hă,ha,hä,ha,ha,he
1,ሁ,1201,hu,hu,hu,hū,hu,hu,hu,hu,hu,hu
2,ሂ,1202,hi,hī,hi,hī,hi,hi,hi,hi,hi,hi
3,ሃ,1203,hā,hā,ha,hā,ha,ha,ha,ha,hā,ha
4,ሄ,1204,hé,hē,he,hē,he,he,he,he,hē,hE


In [25]:
df.to_csv("raw/transliteration-table.csv", index=False)

# Simplify the transliteration table

In [109]:
df = pd.read_csv("raw/transliteration-table.csv")
df = df.drop(columns=['USV'])
df = df.set_index('Glyph')
df

,ALA-LC,Alone-Stokes,B&D,Campbell,Chaîne,Cohen,Dawkins,Dept of State,ethiop,SERA
Glyph,,,,,,,,,,
ሀ,ha,hā,hä,ha,hă,ha,hä,ha,ha,he
ሁ,hu,hu,hu,hū,hu,hu,hu,hu,hu,hu
ሂ,hi,hī,hi,hī,hi,hi,hi,hi,hi,hi
ሃ,hā,hā,ha,hā,ha,ha,ha,ha,hā,ha
ሄ,hé,hē,he,hē,he,he,he,he,hē,hE
...,...,...,...,...,...,...,...,...,...,...
ᎎ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pwē,pWE
ᎏ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pwe,pW
ፘ,rya,NaN,NaN,NaN,NaN,rya,rya,NaN,~ri,NaN


In [110]:
import unidecode
import numpy as np

def simplify(value):
    try:
        return unidecode.unidecode(value).lower()
    except:
        return np.nan
    
simplified = df.applymap(simplify)
merged = df.join(simplified, rsuffix='_simp')
merged.head(10)

,ALA-LC,Alone-Stokes,B&D,Campbell,Chaîne,Cohen,Dawkins,Dept of State,ethiop,SERA,ALA-LC_simp,Alone-Stokes_simp,B&D_simp,Campbell_simp,Chaîne_simp,Cohen_simp,Dawkins_simp,Dept of State_simp,ethiop_simp,SERA_simp
Glyph,,,,,,,,,,,,,,,,,,,,
ሀ,ha,hā,hä,ha,hă,ha,hä,ha,ha,he,ha,ha,ha,ha,ha,ha,ha,ha,ha,he
ሁ,hu,hu,hu,hū,hu,hu,hu,hu,hu,hu,hu,hu,hu,hu,hu,hu,hu,hu,hu,hu
ሂ,hi,hī,hi,hī,hi,hi,hi,hi,hi,hi,hi,hi,hi,hi,hi,hi,hi,hi,hi,hi
ሃ,hā,hā,ha,hā,ha,ha,ha,ha,hā,ha,ha,ha,ha,ha,ha,ha,ha,ha,ha,ha
ሄ,hé,hē,he,hē,he,he,he,he,hē,hE,he,he,he,he,he,he,he,he,he,he
ህ,he/h,h/hi,hə/hø,he,hĕ,h/hə,hï,hɪ,he,h,he/h,h/hi,h@/ho,he,he,h/h@,hi,hi,he,h
ሆ,ho,ho,ho,hō,ho,hʷo,ho,ho,ho,ho,ho,ho,ho,ho,ho,hwo,ho,ho,ho,ho
ለ,la,la,lä,la,lă,lɑ,lä,lə,la,le,la,la,la,la,la,la,la,l@,la,le
ሉ,lu,lu,lu,lū,lu,lu,lu,lu,lu,lu,lu,lu,lu,lu,lu,lu,lu,lu,lu,lu


In [134]:
def split_apart(key):
    return merged[key].str.split("/", expand=True).add_suffix('_' + key)

# Split apart he/h into two columns
colnames = list(merged.columns)
split = pd.concat([split_apart(col) for col in colnames], axis=1)
split

,0_ALA-LC,1_ALA-LC,0_Alone-Stokes,1_Alone-Stokes,0_B&D,1_B&D,2_B&D,3_B&D,0_Campbell,0_Chaîne,...,0_Campbell_simp,0_Chaîne_simp,0_Cohen_simp,1_Cohen_simp,2_Cohen_simp,0_Dawkins_simp,0_Dept of State_simp,0_ethiop_simp,0_SERA_simp,1_SERA_simp
Glyph,,,,,,,,,,,,,,,,,,,,,
ሀ,ha,None,hā,None,hä,None,None,None,ha,hă,...,ha,ha,ha,None,None,ha,ha,ha,he,None
ሁ,hu,None,hu,None,hu,None,None,None,hū,hu,...,hu,hu,hu,None,None,hu,hu,hu,hu,None
ሂ,hi,None,hī,None,hi,None,None,None,hī,hi,...,hi,hi,hi,None,None,hi,hi,hi,hi,None
ሃ,hā,None,hā,None,ha,None,None,None,hā,ha,...,ha,ha,ha,None,None,ha,ha,ha,ha,None
ሄ,hé,None,hē,None,he,None,None,None,hē,he,...,he,he,he,None,None,he,he,he,he,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ᎎ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pwe,pwe,None
ᎏ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pwe,pw,None
ፘ,rya,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,rya,None,None,rya,NaN,~ri,NaN,NaN


In [162]:
uniqued = df.fillna('').apply(lambda row: list(set([v for v in row.values if '/' not in str(v)])), axis=1)
uniqued

Glyph
ሀ    [hă, hä, he, hā, ha]
ሁ                [hu, hū]
ሂ                [hi, hī]
ሃ                [hā, ha]
ሄ        [hé, hE, hē, he]
             ...         
ᎎ            [, pwē, pWE]
ᎏ             [, pwe, pW]
ፘ            [, ~ri, rya]
ፙ            [, ~mA, mya]
ፚ            [, ~fi, fya]
Length: 368, dtype: object

In [166]:
uniqued.to_json("raw/lookup-table.json")

In [167]:
#merged.to_csv("raw/transliteration-table-large.csv")

# Use to transliterate

In [287]:
import json

with open("raw/lookup-table.json") as f:
    lookup = json.load(f)

lookup_simp = {}
for key in lookup.keys():
    values = list(set([unidecode.unidecode(v).lower() for v in lookup[key]]))
    lookup_simp[key] = [v for v in values if '@' not in v]
    
lookup_simp

{'ሀ': ['he', 'ha'],
 'ሁ': ['hu'],
 'ሂ': ['hi'],
 'ሃ': ['ha'],
 'ሄ': ['he'],
 'ህ': ['h', 'hi', 'he'],
 'ሆ': ['hwo', 'ho'],
 'ለ': ['la', 'le'],
 'ሉ': ['lu'],
 'ሊ': ['li'],
 'ላ': ['la'],
 'ሌ': ['lye', 'le'],
 'ል': ['le', 'li', 'l'],
 'ሎ': ['lo', 'lwo'],
 'ሏ': ['', 'lwa'],
 'ሐ': ['he', 'ha'],
 'ሑ': ['hu'],
 'ሒ': ['hi'],
 'ሓ': ['ha'],
 'ሔ': ['he'],
 'ሕ': ['hi', 'h', 'he'],
 'ሖ': ['hwo', 'ho'],
 'ሗ': ['', 'hwa'],
 'መ': ['ma', 'me'],
 'ሙ': ['mu'],
 'ሚ': ['mi'],
 'ማ': ['ma'],
 'ሜ': ['me', 'mye'],
 'ም': ['mi', 'me', 'm'],
 'ሞ': ['mwo', 'mo'],
 'ᎀ': ['', 'mwe', 'mwa'],
 'ᎁ': ['', 'mwi'],
 'ሟ': ['', 'mwa'],
 'ᎂ': ['', 'mwe'],
 'ᎃ': ['', 'mwe', 'mw'],
 'ሠ': ["'se", 'sa'],
 'ሡ': ['su', "'su"],
 'ሢ': ['si', "'si"],
 'ሣ': ["'sa", 'sa'],
 'ሤ': ['se', 'sye', "'se"],
 'ሥ': ['se', "'s", 'si'],
 'ሦ': ["'so", 'swo', 'so'],
 'ሧ': ['', "'swa", 'swa'],
 'ረ': ['ra', 're'],
 'ሩ': ['ru'],
 'ሪ': ['ri'],
 'ራ': ['ra'],
 'ሬ': ['re', 'rye'],
 'ር': ['ri', 're', 'r'],
 'ሮ': ['ro', 'rwo'],
 'ሯ': ['', 'rwa'],
 'ሰ': ['se'

In [288]:
import random

def select_char(c, simple=False):
    try:
        if simple:
            return random.choice(lookup_simp[c])
        else:
            return random.choice(lookup[c])
    except:
        # Probably punctuation, we need to deal with that!
        return " "

def transliterate(word, simple=False):
    return ''.join([select_char(c, simple=simple) for c in word]).replace("  ", " ")

Every time you run it, a different transcription

In [289]:
transliterate("""እኔም፥ ወንድሞች ሆይ፥ የሥጋ እንደ መሆናችሁ፥ 
በክርስቶስም ሕፃናት እንደ መሆናችሁ እንጂ መንፈሳውያን እንደ 
መሆናችሁ ልናገራችሁ አልቻልሁም።""", simple=True)

"`enyem wenidimochi hoyi jasega 'enda mehonahu  bakersetosime hi'sana'ti inde mahonatchehu inji menfesawejane 'enida mehonacihu lna'geratchehu alilhume "

In [290]:
transliterate("""እኔም፥ ወንድሞች ሆይ፥ የሥጋ እንደ መሆናችሁ፥ 
በክርስቶስም ሕፃናት እንደ መሆናችሁ እንጂ መንፈሳውያን እንደ 
መሆናችሁ ልናገራችሁ አልቻልሁም።""", simple=True)

"'enyemi wenedmotche hwoy ya'sga inida mahwonahu  bakresetosim hecanat `enede mahona'cehu ineji menifasawijan `enda mahwonatchehu linagarahu 'altchalehumi "

In [291]:
transliterate("""እኔም፥ ወንድሞች ሆይ፥ የሥጋ እንደ መሆናችሁ፥ 
በክርስቶስም ሕፃናት እንደ መሆናችሁ እንጂ መንፈሳውያን እንደ 
መሆናችሁ ልናገራችሁ አልቻልሁም።""", simple=True)

"'enyeme wandemwochi hoye yasiga 'enide mahwona'hu  bekirisitosmi hisana'ti 'enida mahona'hu `endji manefasawyan `enide mahonahu lna'garacihu `alecalihume "

In [292]:
transliterate("""እኔም፥ ወንድሞች ሆይ፥ የሥጋ እንደ መሆናችሁ፥ 
በክርስቶስም ሕፃናት እንደ መሆናችሁ እንጂ መንፈሳውያን እንደ 
መሆናችሁ ልናገራችሁ አልቻልሁም።""", simple=True)

"ineme wenedmwotche hwoyi yasega `eneda mehona'chu  bekeresetosim hsanati 'enida mahwonatchehu in manefasawiyani inede mahwona'cehu lna'garahu 'alichalehum "

In [293]:
transliterate("""እኔም፥ ወንድሞች ሆይ፥ የሥጋ እንደ መሆናችሁ፥ 
በክርስቶስም ሕፃናት እንደ መሆናችሁ እንጂ መንፈሳውያን እንደ 
መሆናችሁ ልናገራችሁ አልቻልሁም።""", simple=True)

"`enemi wendmwoce hwoje ja'sga inida mehona'cihu  bekiresitosemi hecana't 'enide mehwonachihu `eni manifasawejan inida mehwona'tchehu lnagaratchehu alecalhume "

In [294]:
transliterate("""እኔም፥ ወንድሞች ሆይ፥ የሥጋ እንደ መሆናችሁ፥ 
በክርስቶስም ሕፃናት እንደ መሆናችሁ እንጂ መንፈሳውያን እንደ 
መሆናችሁ ልናገራችሁ አልቻልሁም።""", simple=True)

"`enyemi wandemwochi hwoy ja'sga `enede mehwonachihu  bekerisitwosmi h'sanate inda mahwonacihu inji menfesawyani `eneda mahona'chu lnagaracehu 'alichalihume "

`Simple=False` to allow accents and `@`

In [295]:
transliterate("""እኔም፥ ወንድሞች ሆይ፥ የሥጋ እንደ መሆናችሁ፥ 
በክርስቶስም ሕፃናት እንደ መሆናችሁ እንጂ መንፈሳውያን እንደ 
መሆናችሁ ልናገራችሁ አልቻልሁም።""", simple=False)

'ɪnēm wånɪdĕmʷotchĕ hōyï yesɪga ïnde măhonāčehū  bɑkresïtōsĕm hï‘Sanātɪ ʼenïdɑ məhōnachïhū ənĕji mɑnefɑsāwɪjāne əneda mahōnāʼčehū lɪnāʼgărahu ʼalïtchalehumɪ '